In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
import glob
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean

from tqdm.notebook import tqdm
from pycaret.regression import *
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# 스마트팜 배출량에 대한 예측
- smart_farm_final.csv

# Pycaret을 이용한 튜닝 및 예측
 - 3 표준편차 기준 이상치 대체 반응변수
 - 왜도 3 이상 값들 로그변환 설명변수
 - VIF < 11 인 값들
 - 부스팅 / 배깅 / 블랜딩 3개 혼합모델

In [108]:
#data = pd.read_csv("smart_farm_final.csv").sample(n = 50000)
data = pd.read_csv("smart_farm_final.csv")
data.shape

(148168, 57)

In [104]:
data["환기온도(천창 제어온도)"].value_counts()

28    79025
30    41149
27    27994
Name: 환기온도(천창 제어온도), dtype: int64

In [51]:
data.iloc[129448:,:]

,시간,년도,월,일,시,분,일사량(W/㎡),외기온(℃),외기습도(%),풍속(m/s),...,하루길이,시간범주,일몰전후,일출일몰,지면온도,오존등급,아황산가스등급,이산화질소등급,PM10등급,PM25등급
129448,2022-03-04 00:00:00,2022,3,4,0,0,0.0,2.0,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129449,2022-03-04 00:01:00,2022,3,4,0,1,0.0,2.0,76,0.4,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129450,2022-03-04 00:02:00,2022,3,4,0,2,0.0,1.9,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129451,2022-03-04 00:03:00,2022,3,4,0,3,0.0,1.9,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
129452,2022-03-04 00:04:00,2022,3,4,0,4,0.0,1.9,76,0.0,...,1131,야간,일몰1시간반_후,일몰,-1.8,보통,좋음,좋음,보통,보통
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148163,2022-03-20 23:55:00,2022,3,20,23,55,0.0,6.2,61,0.4,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음
148164,2022-03-20 23:56:00,2022,3,20,23,56,0.0,6.2,61,0.0,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음
148165,2022-03-20 23:57:00,2022,3,20,23,57,0.0,6.1,61,0.4,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음
148166,2022-03-20 23:58:00,2022,3,20,23,58,0.0,6.1,61,0.4,...,1207,야간,일몰1시간반_후,일몰,4.6,보통,좋음,좋음,좋음,좋음


시간,년월일 제거

In [52]:
data = data.drop(["시간","년월일","년도","월","일","시","분"],axis=1)

In [64]:
train = data.iloc[:129448,:]
test = data.iloc[129448:,:]

문자열 칼럼 추출

In [65]:
# 문자열 형식의 컬럼만 추출하기
object_lst = []

# 숫자 형식의 컬럼만 추출하기
num_lst = []

for col in data.columns:
    if data[col].dtype == "object":
        object_lst.append(col)
    
    else:
        num_lst.append(col)

In [66]:
# 숫자관련

#  '배기팬','유동팬','팬코일 B동','환기온도(천창 제어온도)', "차광스크린_개방" 은 categorical

num_lst

['일사량(W/㎡)',
 '외기온(℃)',
 '외기습도(%)',
 '풍속(m/s)',
 '풍향(º)',
 '내부 온도',
 '내부 습도',
 '차광스크린(수평)',
 '차열스크린(수평)',
 '에너지스크린(수평)',
 '에너지스크린(수직)',
 '배기팬',
 '천창',
 '유동팬',
 '팬코일 B동',
 '환기온도(천창 제어온도)',
 '난방온도(설정온도)',
 '누적강수량',
 '현지기압',
 '해면기압',
 '일조',
 'PM10_1시간',
 'PM25_1시간',
 '오존_1시간',
 '이산화질소_1시간',
 '일산화탄소_1시간',
 '아황산가스_1시간',
 '차광스크린_개방',
 '일출시간',
 '일몰시간',
 '일출3시간후',
 '일출1시간후',
 '일몰3시간후',
 '일몰1시간반전',
 '하루길이',
 '지면온도',
 '열공급량(kWh)']

In [55]:
# 카테고리열 제거한 numerical

num_lst_real = [x for x in num_lst if x not in ['배기팬','유동팬','팬코일 B동','환기온도(천창 제어온도)', "차광스크린_개방","열공급량(kWh)"]]

object_lst_real = object_lst + ['배기팬','유동팬','팬코일 B동','환기온도(천창 제어온도)', "차광스크린_개방"]

num_lst_real

['일사량(W/㎡)',
 '외기온(℃)',
 '외기습도(%)',
 '풍속(m/s)',
 '풍향(º)',
 '내부 온도',
 '내부 습도',
 '차광스크린(수평)',
 '차열스크린(수평)',
 '에너지스크린(수평)',
 '에너지스크린(수직)',
 '천창',
 '난방온도(설정온도)',
 '누적강수량',
 '현지기압',
 '해면기압',
 '일조',
 'PM10_1시간',
 'PM25_1시간',
 '오존_1시간',
 '이산화질소_1시간',
 '일산화탄소_1시간',
 '아황산가스_1시간',
 '일출시간',
 '일몰시간',
 '일출3시간후',
 '일출1시간후',
 '일몰3시간후',
 '일몰1시간반전',
 '하루길이',
 '지면온도']

In [56]:
object_lst_real

['차광스크린_광도조절',
 '차광스크린_개방조절',
 '차열스크린_온도조절',
 '차열스크린_광도조절',
 '차열스크린_개방조절',
 '시간범주',
 '일몰전후',
 '일출일몰',
 '오존등급',
 '아황산가스등급',
 '이산화질소등급',
 'PM10등급',
 'PM25등급',
 '배기팬',
 '유동팬',
 '팬코일 B동',
 '환기온도(천창 제어온도)',
 '차광스크린_개방']

In [57]:
reg_lunch=setup(
        data=data,
        target='열공급량(kWh)',
        train_size=0.8,
        session_id = 0,
        imputation_type='simple',
        data_split_shuffle=False,
        normalize = True,
        normalize_method = 'zscore',
        fold_strategy = 'timeseries',
        fold = 5,
        verbose=True,
        silent= True, n_jobs= 1,
        categorical_features = object_lst_real,
        numeric_features = num_lst_real)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:46:19
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\kimhy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-57-9cde9687e1fd>", line 1, in <module>
    reg_lunch=setup(
  File "C:\Users\kimhy\AppData\Roaming\Python\Python38\site-packages\pycaret\regression.py", line 573, in setup
    return pycaret.internal.tabular.setup(
  File "C:\Users\kimhy\AppData\Roaming\Python\Python38\site-packages\pycaret\internal\tabular.py", line 1350, in setup
    data = prep_pipe.transform(data_before_preprocess.copy())
  File "C:\Users\kimhy\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 549, in _transform
    Xt = transform.transform(Xt)
  File "C:\Users\kimhy\AppData\Roaming\Python\Python38\site-packages\pycaret\internal\preprocess.py", line 370, in transform
    data.replace([np.inf, -np.inf], np.NaN, inplace=True)
  File "C:\Users\kimhy\anaconda3\lib\site-packages\p

TypeError: object of type 'NoneType' has no len()

그리 높지 않은 성능

In [12]:
best_5_d = compare_models(sort = 'RMSE', n_select = 5,round=4,cross_validation=True,verbose=True)
print(best_5_d)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.1166,0.0473,0.2175,0.5204,0.1446,0.4911,16.3080
lightgbm,Light Gradient Boosting Machine,0.1362,0.0487,0.2205,0.5069,0.1480,0.5471,0.3140
et,Extra Trees Regressor,0.1187,0.0529,0.2298,0.4642,0.1526,0.4909,7.7460
gbr,Gradient Boosting Regressor,0.1553,0.0557,0.2359,0.4362,0.1598,0.5966,3.7740
knn,K Neighbors Regressor,0.1375,0.0590,0.2427,0.4024,0.1626,0.5699,4.1100
lr,Linear Regression,0.1832,0.0707,0.2659,0.2835,0.1803,0.6890,0.0280
ridge,Ridge Regression,0.1832,0.0707,0.2659,0.2835,0.1803,0.6888,0.0160
br,Bayesian Ridge,0.1830,0.0707,0.2659,0.2840,0.1802,0.6876,0.0600
omp,Orthogonal Matching Pursuit,0.1834,0.0722,0.2687,0.2684,0.1821,0.6877,0.0140
huber,Huber Regressor,0.1757,0.0741,0.2723,0.2491,0.1806,0.6175,1.1960


[RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=1, num_leaves=31, objective=None,
              random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0), ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, crite

In [148]:
reg_setup = setup(data = train, test_data = test, target = '열공급량(kWh)', fold_strategy = 'timeseries',
                  fold = 5, session_id = 0,
                  categorical_features = object_lst_real, n_jobs= 1,
                  numeric_features = num_lst_real, normalize = False, verbose = False)

Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
일사량(W/㎡),Numeric
외기온(℃),Numeric
외기습도(%),Numeric
풍속(m/s),Numeric
풍향(º),Numeric
내부 온도,Numeric
내부 습도,Numeric
차광스크린(수평),Numeric
차열스크린(수평),Numeric
에너지스크린(수평),Numeric


ValueError: Found array with 0 sample(s) (shape=(0, 31)) while a minimum of 1 is required.

--------------------- 구분선 ------------------------------

### 변수 선택

In [67]:
## 전진 단계별 선택법
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

def get_stepwise(df):
    variables = df.columns[:-1].tolist() ## 설명 변수 리스트

    y = df['열공급량(kWh)'] ## 반응 변수
    selected_variables = [] ## 선택된 변수들
    sl_enter = 0.05
    sl_remove = 0.05

    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    while len(variables) > 0:
        remainder = list(set(variables) - set(selected_variables))
        pval = pd.Series(index=remainder) ## 변수의 p-value
        ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
        ## 선형 모형을 적합한다.
        for col in remainder: 
            X = df[selected_variables+[col]]
            X = sm.add_constant(X)
            model = sm.OLS(y,X).fit()
            pval[col] = model.pvalues[col]

        min_pval = pval.min()
        if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
            selected_variables.append(pval.idxmin())
            ## 선택된 변수들에대해서
            ## 어떤 변수를 제거할지 고른다.
            while len(selected_variables) > 0:
                selected_X = df[selected_variables]
                selected_X = sm.add_constant(selected_X)
                selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
                max_pval = selected_pval.max()
                if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                    remove_variable = selected_pval.idxmax()
                    selected_variables.remove(remove_variable)
                else:
                    break

            step += 1
            steps.append(step)
            
            adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(selected_variables.copy())
            print(adj_r_squared,"스텝",step)
            if step > 373:
                break
        else:
            break
    return selected_variables,sv_per_step,adjusted_r_squared,steps

In [70]:
def make_df(data):
    df = data.copy()
    y = df["열공급량(kWh)"]
    df =  pd.get_dummies(df.drop(["열공급량(kWh)"],axis=1),
                         columns = object_lst_real)

    #df = sm.add_constant(df, has_constant = "add")
    df["열공급량(kWh)"] = y # 끝에 붙여주기
    df.head(3)
    return df

In [71]:
a,b,c,d = get_stepwise(make_df(train))

0.04167675842676544 스텝 1
0.15650986867119632 스텝 2
0.16678769110662195 스텝 3
0.16678769110662506 스텝 4
0.23796150278707673 스텝 5
0.25594441597037565 스텝 6
0.25594441597037554 스텝 7
0.26056139774146936 스텝 8
0.2605613977414696 스텝 9
0.26831121681267167 스텝 10
0.27232458335579335 스텝 11
0.2745920398321654 스텝 12
0.2770548568032033 스텝 13
0.2784985866696813 스텝 14
0.28035685089411144 스텝 15
0.2830505715971523 스텝 16
0.2838647064036497 스텝 17
0.284884705122488 스텝 18
0.285519326171226 스텝 19
0.2855193261712262 스텝 20
0.28613734889381526 스텝 21
0.28671783955915264 스텝 22
0.2870460501151588 스텝 23
0.28704605011515893 스텝 24
0.28725337580153676 스텝 25
0.28725337580153665 스텝 26
0.2874610657817006 스텝 27
0.28769378221949937 스텝 28
0.28786782739201133 스텝 29
0.2880905472314831 스텝 30
0.288090547231483 스텝 31
0.2884607697327475 스텝 32
0.28873246500992344 스텝 33
0.2892485645145537 스텝 34
0.289461358148953 스텝 35
0.2896618993874094 스텝 36
0.28966189938740927 스텝 37
0.28984463744563704 스텝 38
0.2900076624844785 스텝 39
0.290131683490216

MemoryError: Unable to allocate 41.5 MiB for an array with shape (42, 129448) and data type float64

In [73]:
make_df(data).columns

Index(['일사량(W/㎡)', '외기온(℃)', '외기습도(%)', '풍속(m/s)', '풍향(º)', '내부 온도', '내부 습도',
       '차광스크린(수평)', '차열스크린(수평)', '에너지스크린(수평)', '에너지스크린(수직)', '천창',
       '난방온도(설정온도)', '누적강수량', '현지기압', '해면기압', '일조', 'PM10_1시간', 'PM25_1시간',
       '오존_1시간', '이산화질소_1시간', '일산화탄소_1시간', '아황산가스_1시간', '일출시간', '일몰시간',
       '일출3시간후', '일출1시간후', '일몰3시간후', '일몰1시간반전', '하루길이', '지면온도',
       '차광스크린_광도조절_1000미만', '차광스크린_개방조절_0%~30%개방', '차광스크린_개방조절_100%개방',
       '차광스크린_개방조절_30%~80%개방', '차광스크린_개방조절_80%~100%개방', '차열스크린_온도조절_23도~25도',
       '차열스크린_온도조절_23도미만', '차열스크린_온도조절_25도~26도', '차열스크린_온도조절_26도이상',
       '차열스크린_광도조절_800미만', '차열스크린_개방조절_0%~30%개방', '차열스크린_개방조절_100%개방',
       '차열스크린_개방조절_30%~70%개방', '차열스크린_개방조절_70%~100%개방', '시간범주_그외', '시간범주_야간',
       '시간범주_주간', '일몰전후_일몰1시간반_전', '일몰전후_일몰1시간반_후', '일출일몰_일몰', '일출일몰_일출',
       '오존등급_보통', '오존등급_좋음', '아황산가스등급_보통', '아황산가스등급_좋음', '이산화질소등급_나쁨',
       '이산화질소등급_보통', '이산화질소등급_좋음', 'PM10등급_나쁨', 'PM10등급_보통', 'PM10등급_좋음',
       'PM25등급_나쁨', 'PM25등급_보통', 'PM25등급_좋음', '배기팬_0', '

변수선택..제발

## 변수 선택 이후 스케일링, 로그변환, 이상치 처리

#### 이상치 대체하기

In [105]:
def replace_outlier_zscore(df,columns,n):
    data = df.copy()
    mean = np.mean(data[columns])
    std = np.std(data[columns])
    score_top = int(n)*std + mean
    score_bot = -1*n*std + mean
    
    def cond1(x):
        if x > score_top:
            return score_top
        if x < score_bot:
            return score_bot
        else:
            return x
    data[columns] = data[columns].apply(cond1)
    
    return data[columns]

In [117]:
# data

for col in ['일사량(W/㎡)','외기온(℃)','외기습도(%)','풍속(m/s)', '풍향(º)','내부 온도','내부 습도','차광스크린(수평)','차열스크린(수평)',
 '에너지스크린(수평)','에너지스크린(수직)','천창','난방온도(설정온도)','누적강수량','현지기압','해면기압','PM10_1시간','PM25_1시간',
 '오존_1시간','이산화질소_1시간','일산화탄소_1시간','아황산가스_1시간','지면온도']:
    
    data[col+"_scale"] = data[col]  

In [133]:
for col in ['일사량(W/㎡)','외기온(℃)','외기습도(%)','풍속(m/s)', '풍향(º)','내부 온도','내부 습도','차광스크린(수평)','차열스크린(수평)',
 '에너지스크린(수평)','에너지스크린(수직)','천창','난방온도(설정온도)','누적강수량','현지기압','해면기압','PM10_1시간','PM25_1시간',
 '오존_1시간','이산화질소_1시간','일산화탄소_1시간','아황산가스_1시간','지면온도']:
    
    data[col+"_scale"] = replace_outlier_zscore(data,col+"_scale",3)
    
    if (data[col] == data[col+"_scale"]).sum() == 148168:
        pass
    else:
        print(col)
        print("개수 : ",148168 - (data[col] == data[col+"_scale"]).sum())
        
    data.drop([col+"_scale"],axis=1,inplace=True)

일사량(W/㎡)
개수 :  526
외기온(℃)
개수 :  334
풍속(m/s)
개수 :  3456
내부 온도
개수 :  101
내부 습도
개수 :  1364
천창
개수 :  20586
누적강수량
개수 :  5589
현지기압
개수 :  961
해면기압
개수 :  933
PM10_1시간
개수 :  2387
PM25_1시간
개수 :  2745
오존_1시간
개수 :  300
이산화질소_1시간
개수 :  742
일산화탄소_1시간
개수 :  2160
아황산가스_1시간
개수 :  148168
지면온도
개수 :  4380


In [96]:
(data["내부 온도"] == data["내부 온도_scale"]).sum()

148168

왜도 처리까지

In [127]:
loglist=[]
# 왜도 3인 설명변수 log1p 변환
for val in ['일사량(W/㎡)','외기온(℃)','외기습도(%)','풍속(m/s)', '풍향(º)','내부 온도','내부 습도','차광스크린(수평)','차열스크린(수평)',
 '에너지스크린(수평)','에너지스크린(수직)','천창','난방온도(설정온도)','누적강수량','현지기압','해면기압','PM10_1시간','PM25_1시간',
 '오존_1시간','이산화질소_1시간','일산화탄소_1시간','아황산가스_1시간','지면온도']:
    if abs(data[val].skew())>3:
        data[val] = np.log1p(data[val])
        loglist.append(val)
loglist

['천창', '누적강수량', '아황산가스_1시간']

#### 1. 숫자형 데이터 스케일링 진행
- 컬럼별로 스케일링 진행

In [136]:
from sklearn.preprocessing import StandardScaler

In [139]:
for val in ['일사량(W/㎡)','외기온(℃)','외기습도(%)','풍속(m/s)', '풍향(º)','내부 온도','내부 습도','차광스크린(수평)','차열스크린(수평)',
 '에너지스크린(수평)','에너지스크린(수직)','천창','난방온도(설정온도)','누적강수량','현지기압','해면기압','PM10_1시간','PM25_1시간',
 '오존_1시간','이산화질소_1시간','일산화탄소_1시간','아황산가스_1시간','지면온도']:
    
    scaler = StandardScaler()
    
    
    data_scaled = scaler.fit_transform(data[val].values.reshape(-1,1))
    data[val] = data_scaled

In [7]:
data_scaled = data.copy()

# 날짜 관련 데이터는 스케일링 진행하지 않기 # 카테고리열 데이터
num_lst_nodate = [x for x in num_lst if x not in ["년도","월","일","시","분",
                                                  "배기팬","유동팬","팬코일 B동","환기온도(천창 제어온도)","열공급량(kWh)"]]

loglist=[]
# 왜도 3인 설명변수 log1p 변환
for val in num_lst_nodate:
    if abs(data_scaled[val].skew())>3:
        data_scaled[val] = np.log1p(data_scaled[val])
        loglist.append(val)
loglist


from sklearn.preprocessing import StandardScaler

for col in num_lst_nodate:
    std = StandardScaler()

    data_scaled[col] = std.fit_transform(data_scaled[col].values.reshape(-1,1))

"배기팬","유동팬","팬코일 B동","환기온도(천창 제어온도)" 카테고리열 데이터로 스케일링 필요없음

#### 1.1 데이터 처리하기
- 학습용 데이터 : ~2022.03
- 예측용 데이터 : 2022.03~

X 데이터 처리하기
- dummy 처리
- 왜도 처리

In [8]:
Y = data_scaled["열공급량(kWh)"]

X_dummy = pd.get_dummies(data_scaled.drop(["년월일","열공급량(kWh)"],axis=1)
                         ,columns=["년도"])

In [9]:
data_scaled_final = pd.concat([X_dummy, Y], axis = 1)

train_data = data_scaled_final.iloc[:129448,:]

test_data = data_scaled_final.iloc[129448:,:]

In [10]:
x_train = train_data.drop(["열공급량(kWh)"],axis=1)
y_train = train_data["열공급량(kWh)"]


x_test = test_data.drop(["열공급량(kWh)"],axis=1)
y_test = test_data["열공급량(kWh)"]

In [11]:
reg = setup(data=train_data,
                target='열공급량(kWh)',
                numeric_imputation = 'mean',
                normalize = False,
                normalize_method = 'zscore',
                categorical_features = ['차광스크린_광도조절',"환기온도(천창 제어온도)",
                                   '차광스크린_개방조절','차열스크린_온도조절','차열스크린_광도조절','차열스크린_개방조절',
                                   '시간범주','일몰전후','일출일몰','오존등급','아황산가스등급','이산화질소등급','PM10등급','PM25등급'],
                silent= True,
                fold_strategy = 'timeseries',
                fold = 3,
                session_id = 0,
                verbose=True)

,Description,Value
0,session_id,0
1,Target,열공급량(kWh)
2,Original Data,"(129448, 57)"
3,Missing Values,False
4,Numeric Features,36
5,Categorical Features,19
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(90613, 105)"


단일 모델 선택하기 위한 성능 비교

In [ ]:
best_5_d = compare_models(sort = 'RMSLE', n_select = 5,round=4,cross_validation=True,verbose=True,errors='ignore')
print(best_5_d)

IntProgress(value=0, description='Processing: ', max=89)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:09:31
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
